# Word Embeddings

## TF-IDF Vectorizer

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# Load preprocessed data
df = pd.read_csv('../data/preprocessed/preprocessed_data.csv').astype(str)

In [3]:
df.head()

,Content
0,ariv favorit brandsget appour visionpartnershi...
1,r ckabwickl lebensversicher hol doppelt leb re...
2,stark versicher f r lifestyl held de hom produ...
3,restub airbag freedom safety wat restub skip c...
4,curevac revolutioni mrna f r leb mensch ber be...


In [7]:
# Initialize TF-IDF Vectorizer from sklearn
# Limit to 1000 features for now
vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,3), norm='l2', 
                             max_features=1000)

# Apply vectorizer
tfidf_matrix = vectorizer.fit_transform(df['Content'])

# Assign the transformed data to the 'Content' column
df['Content'] = tfidf_matrix.toarray()

In [8]:
# Tf-idf-weighted document-term matrix sorted by tf-idf scores
df.sort_values(by='Content', ascending=False)

,Content
907,0.758699
504,0.565893
1408,0.520715
1991,0.418213
2694,0.387708
...,...
1234,0.000000
1235,0.000000
1238,0.000000
1239,0.000000


## Word2Vec

In [10]:
# Import word2vec from gensim
from gensim.models import Word2Vec

In [11]:
df['Text'] = pd.read_csv('../data/preprocessed/preprocessed_data.csv').astype(str)

In [13]:
# Train Word2Vec model
word2vec = Word2Vec(df['Text'], window=5, min_count=1)

# Access word embeddings
embeddings = word2vec.wv
print(embeddings)

KeyedVectors<vector_size=100, 27 keys>
